In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [ ]:
features = encoded_dataframe.drop(columns=['LeaveOrNot'])
target = encoded_dataframe['LeaveOrNot']
features_and_targets = [features, target]	

In [ ]:
[X, y] = features_and_targets
X = X.values
y = y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.LongTensor(y_train)
y_test_tensor = torch.LongTensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
batch_size = 165 # You can adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]

In [ ]:
def __init__(self, input_features=10, hidden1=80, hidden2=80, output_features=1):
    super(Salary_Predictor, self).__init__()
    self.fc1 = nn.Linear(input_features, hidden1)
    self.fc2 = nn.Linear(hidden1, hidden2)
    self.out = nn.Linear(hidden2, output_features)

def forward(self, x):
    #x = x.requires_grad_()
    x = F.leaky_relu(self.fc1(x))
    x = F.leaky_relu(self.fc2(x))
    '''x = F.sigmoid(self.out(x))
    x = (x >= 0.5).float().requires_grad_()'''
    x = self.out(x)
    #x = x.long()

    return x

In [ ]:
def model_loss_function():
    '''
	Purpose:
	---
	To define the loss function for the model. Loss function measures 
	how well the predictions of a model match the actual target values 
	in training data.
	
	Input Arguments:
	---
	None

	Returns:
	---
	`loss_function`: This can be a pre-defined loss function in PyTorch
					or can be user-defined

	Example call:
	---
	loss_function = model_loss_function()
	'''
    loss_function = nn.MSELoss()
    
    return loss_function

In [ ]:
def model_optimizer(model):
    '''
	Purpose:
	---
	To define the optimizer for the model. Optimizer is responsible 
	for updating the parameters (weights and biases) in a way that 
	minimizes the loss function.
	
	Input Arguments:
	---
	`model`: An object of the 'Salary_Predictor' class

	Returns:
	---
	`optimizer`: Pre-defined optimizer from Pytorch

	Example call:
	---
	optimizer = model_optimizer(model)
	'''
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    return optimizer

In [ ]:
number_of_epcohs = 20

In [ ]:
def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    '''
	Purpose:
	---
	All the required parameters for training are passed to this function.

	Input Arguments:
	---
	1. `model`: An object of the 'Salary_Predictor' class
	2. `number_of_epochs`: For training the model
	3. `tensors_and_iterable_training_data`: list containing training and validation data tensors 
											 and iterable dataset object of training tensors
	4. `loss_function`: Loss function defined for the model
	5. `optimizer`: Optimizer defined for the model

	Returns:
	---
	trained_model

	Example call:
	---
	trained_model = training_function(model, number_of_epochs, iterable_training_data, loss_function, optimizer)

	'''
    training_data = TensorDataset(tensors_and_iterable_training_data[0],tensors_and_iterable_training_data[2])
    training_loader = tensors_and_iterable_training_data[4]
    #training_loader = DataLoader(training_data, batch_size=64, shuffle=True)

    #validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True)
    for epoch in range(number_of_epochs):
        running_loss = 0.0
        for i, data in enumerate(training_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # print(i)
            # zero the parameter gradients
            optimizer.zero_grad()
            labels = labels.unsqueeze(1)
            
            # forward + backward + optimize 
            outputs = model(inputs)
            
            # print(outputs)
            loss = loss_function(outputs.float(), labels.float())
            # Backward pass
            loss.backward()
            # Update model parameters
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        average_loss = running_loss / len(training_loader)
        
        # Print epoch-wise loss
        #print(f"Epoch {epoch+1}/{number_of_epochs}: Loss = {average_loss}")
            # if i % 2000 == 1999:    # print every 2000 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #           (epoch + 1, i + 1, running_loss / 2000))
            #     running_loss = 0.0

           
    return model
